# Detecting and Combating Misinformation on Social Media


Social Media has opened a whole new world for people around the globe.People are just a click away from getting huge chunk of information. With information comes people’s opinion and with this comes the positive and negative outlook of people regarding a topic. Sometimes this also results into bullying and passing on hate comments about someone or something.

This "Misinformation" might include hoaxes, conspiracy theories, etc. and can be used to misguide the reader. To combat such nuisance caused by misinformation we must know how to use social media in a critical way and understand the mechanisms of these platforms as nowadays social media presence is also an important tool for Businesses and individuals as well. The model designed can easily classify true and fake news by using twitter dataset for training.

# Importing necessary libraries

In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

## Loading  Data

### Preprocessing the 2 datasets and combining them.

In [86]:
data_fake = pd.read_csv(r"C:\Users\sarth\Downloads\Fake.csv")
data_true = pd.read_csv(r"C:\Users\sarth\Downloads\True.csv")
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


### Assigning Classes to the Dataset

In [87]:
data_fake["class"] = 0
data_true['class'] = 1

### Checking Number of Rows and Columns in the Dataset


In [88]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

### Manual Testing for Both the Dataset

In [89]:
import pandas as pd  # Assuming pandas is imported

data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop(i, axis=0, inplace=True)  # Pass axis=0 explicitly


data_true_manual_testing = data_true.tail(10)
for i in range(21416,214106, -1):
    data_true.drop(i, axis=0, inplace=True)  # Pass axis=0 explicitly

### Assigning Classes to the Dataset


In [57]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\sarth\AppData\Local\Temp/ipykernel_23888/277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\sarth\AppData\Local\Temp/ipykernel_23888/277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


### Merging Both the Dataset

In [8]:
# Concatenate the DataFrames
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


### Dropping Unwanted Columns

In [9]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [10]:
data = data_merge.drop(['title','subject','date'], axis = 1)


In [11]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [12]:
data = data.sample(frac = 1)

In [13]:
data.head()

,text,class
21326,They were probably just looking for a safe sp...,0
1120,GENEVA (Reuters) - U.S. President Donald Trump...,1
20173,GENEVA (Reuters) - Human rights activists call...,1
20735,TALLINN/PARIS (Reuters) - German Chancellor An...,1
7711,"(Reuters) - Forget the accusations of groping,...",1


In [14]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [15]:
data.columns

Index(['text', 'class'], dtype='object')

In [16]:
data.head()

,text,class
0,They were probably just looking for a safe sp...,0
1,GENEVA (Reuters) - U.S. President Donald Trump...,1
2,GENEVA (Reuters) - Human rights activists call...,1
3,TALLINN/PARIS (Reuters) - German Chancellor An...,1
4,"(Reuters) - Forget the accusations of groping,...",1


### Create a Function to Clean Text


In [17]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [18]:
# Applying function to text column
data["text"] = data["text"].apply(wordopt)

### Defining dependent and independent variables

In [19]:
x = data["text"]
y = data["class"]

### Splitting Training and Testing

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

### Converting Raw Data Into Matrix for Further Process.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### Creating First Model using LR 

In [22]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

### Checking the Model Accuracy and Classification Report

In [23]:
pred_lr=LR.predict(xv_test)
LR.score(xv_test, y_test)

0.9875222816399287

In [30]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5832
           1       0.99      0.99      0.99      5388

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### Creating a Second Model Decision Tree Classification.

In [25]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

### Checking the Model Accuracy and Classification Report

In [26]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9958110516934047

In [31]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5832
           1       1.00      0.99      1.00      5388

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Third Model- Gradient Boosting Classifier

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

### Checking the Model Accuracy and Classification Report

In [33]:
pred_gbc = GBC.predict(xv_test)
GBC.score(xv_test, y_test)

0.9961675579322639

In [34]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5832
           1       0.99      1.00      1.00      5388

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



### Fourth Model -  Random Forest Classifier

In [35]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

### Checking the Model Accuracy and Classification Report

In [36]:
pred_rfc = RFC.predict(xv_test)
RFC.score(xv_test, y_test)

0.9868983957219252

In [37]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5832
           1       0.99      0.98      0.99      5388

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



### Model testing Checking fake news

In [62]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [82]:
news = str(input())
manual_testing(news)

The following statementsÂ were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own.Â Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - Together, we are MAKING AMERICA GREAT AGAIN! bit.ly/2lnpKaq [1814 EST] - In the East, it could be the COLDEST New Yearâ€™s Eve on record. Perhaps we could use a little bit of that good old Global Warming that our Country, but not other countries, was going to pay TRILLIONS OF DOLLARS to protect against. Bundle up! [1901 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR) 


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GBC Prediction: Not A Fake News 
RFC Prediction: Not A Fake News


In [81]:
news = str(input())
manual_testing(news)

SEATTLE/WASHINGTON (Reuters) - President Donald Trump called on the U.S. Postal Service on Friday to charge â€œmuch moreâ€ to ship packages for Amazon (AMZN.O), picking another fight with an online retail giant he has criticized in the past.     â€œWhy is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE!â€ Trump wrote on Twitter.  The presidentâ€™s tweet drew fresh attention to the fragile finances of the Postal Service at a time when tens of millions of parcels have just been shipped all over the country for the holiday season.  The U.S. Postal Service, which runs at a big loss, is an independent agency within the federal government and does not receive tax dollars for operating expenses, according to its website.  Package delivery has become an increasingly important part of its business as the 